# Todo and Planning Tools

In this notebook, we'll explore how to use the SDK's task management tools to plan and track multi-step workflows.

## Setup

First, let's configure our environment:

In [ ]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

In [ ]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

## Helper Function

Let's create a helper to display messages:

In [ ]:
import json

def print_message(message):
    """Pretty print agent messages."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        pass
    
    elif msg_type == "AssistantMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"🤖 Assistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"🔧 Tool: {block.name}")
                    if hasattr(block, 'input'):
                        if 'description' in block.input:
                            print(f"   → {block.input['description']}")
                        args = {k: v for k, v in block.input.items() if k != 'description'}
                        if args:
                            print(f"   Arguments: {json.dumps(args, indent=6)}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"❌ Tool Error: {block.content}")
                    else:
                        content = str(block.content)
                        if len(content) > 500:
                            content = content[:500] + "..."
                        print(f"📤 Tool Result: {content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\n💰 Cost: ${message.total_cost_usd:.4f} | ⏱️ Time: {message.duration_ms/1000:.1f}s")

## Why Task Management?

When working on complex multi-step projects, you need to:
- Break work into discrete tasks
- Track progress across steps
- Handle dependencies between tasks
- Coordinate multiple agents or workflow stages

The SDK provides built-in tools for this:

| Tool | Purpose |
|------|------|
| `TaskCreate` | Create new tasks with descriptions |
| `TaskUpdate` | Update status, add dependencies |
| `TaskList` | View all tasks and their status |
| `TaskGet` | Get detailed info about a specific task |

## Example 1: Basic Task Creation

Let's create a simple task and track it:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions

print("=" * 60)
print("Creating and tracking a simple task")
print("=" * 60)

async def create_task_demo():
    async for msg in query(
        prompt="""Create a task to analyze all Python files in the current directory. 
        Then list all tasks to show it was created.""",
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)

await create_task_demo()

### What Happened?

The agent used the `TaskCreate` tool to:
- Create a task with a subject and description
- Set an `activeForm` (displayed when task is in progress)
- Assign initial status as `pending`

Then it used `TaskList` to show the created task.

## Example 2: Multi-Step Workflow with Dependencies

Let's create a more complex workflow with task dependencies:

In [ ]:
print("=" * 60)
print("Creating a multi-step workflow")
print("=" * 60)

async def multi_step_workflow():
    async for msg in query(
        prompt="""Create a 4-step code review workflow:
        
        1. Find all Python files
        2. Read and analyze the first file (depends on step 1)
        3. Check for security issues (depends on step 2)
        4. Generate a summary report (depends on step 3)
        
        Create these tasks with proper dependencies, then list all tasks to show the plan.""",
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)

await multi_step_workflow()

### Task Dependencies

Notice how the agent created tasks with `blockedBy` relationships:
- Task 2 is blocked by Task 1
- Task 3 is blocked by Task 2
- Task 4 is blocked by Task 3

This ensures tasks execute in the correct order.

## Example 3: Updating Task Status

Let's execute tasks and update their status as we progress:

In [ ]:
print("=" * 60)
print("Executing tasks and updating status")
print("=" * 60)

async def execute_tasks():
    # First, create tasks
    async for msg in query(
        prompt="""Create these two tasks:
        1. Count Python files in current directory
        2. Report the count (depends on task 1)
        """,
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)
    
    print("\n" + "=" * 60)
    print("Executing task 1...")
    print("=" * 60)
    
    # Execute task 1 and update status
    async for msg in query(
        prompt="""Update task 1 to 'in_progress', then count the Python files, 
        then update task 1 to 'completed'. Show the task list after.""",
        options=ClaudeAgentOptions(continue_conversation=True)
    ):
        print_message(msg)
    
    print("\n" + "=" * 60)
    print("Executing task 2...")
    print("=" * 60)
    
    # Execute task 2
    async for msg in query(
        prompt="""Now that task 1 is complete, update task 2 to 'in_progress', 
        report the count from before, then mark task 2 as 'completed'.""",
        options=ClaudeAgentOptions(continue_conversation=True)
    ):
        print_message(msg)

await execute_tasks()

### Task Lifecycle

Tasks move through three states:

```
pending → in_progress → completed
```

The agent uses `TaskUpdate` to transition tasks as work progresses.

## Example 4: Real-World Refactoring Workflow

Let's see how to use tasks for a realistic refactoring project:

In [ ]:
print("=" * 60)
print("Planning a refactoring project")
print("=" * 60)

async def refactoring_workflow():
    async for msg in query(
        prompt="""I want to refactor a Python codebase to improve code quality.
        
        Create a comprehensive task plan with these steps:
        1. Identify all Python files that need refactoring
        2. Analyze code for common issues (long functions, duplicated code, etc.)
        3. Prioritize files by complexity/issue count
        4. Refactor high-priority files
        5. Run tests to verify no regressions
        6. Generate refactoring report
        
        Set up proper dependencies so tasks run in order.
        Then show the complete task list.""",
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)

await refactoring_workflow()

### Why This Matters

Breaking complex work into tasks:
- Makes progress visible
- Helps estimate remaining work
- Allows interruption and resumption
- Enables parallel execution (for independent tasks)
- Provides clear accountability

## Example 5: Parallel vs Sequential Tasks

Not all tasks need dependencies. Some can run in parallel:

In [ ]:
print("=" * 60)
print("Creating parallel and sequential tasks")
print("=" * 60)

async def parallel_tasks():
    async for msg in query(
        prompt="""Create this task structure:
        
        Parallel group (can run simultaneously):
        - Task A: Analyze Python files
        - Task B: Analyze markdown files  
        - Task C: Analyze JSON files
        
        Sequential task (depends on all parallel tasks):
        - Task D: Create combined analysis report (blocked by A, B, and C)
        
        Show the task list with dependencies.""",
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)

await parallel_tasks()

### Task Dependency Patterns

```
Sequential:        Parallel:          Fan-in:
A → B → C          A                  A \
                   B                  B  → D
                   C                  C /
```

Use `addBlockedBy` in `TaskUpdate` to create these patterns.

## Example 6: Using TaskGet for Detailed Info

When you need full details about a specific task:

In [ ]:
print("=" * 60)
print("Getting detailed task information")
print("=" * 60)

async def task_details():
    # Create a task first
    async for msg in query(
        prompt="""Create a detailed task with:
        - Subject: 'Implement user authentication'
        - Description: 'Add JWT-based authentication with login/logout endpoints, 
          password hashing, and token refresh. Include unit tests.'
        
        Then use TaskGet to show the full task details.""",
        options=ClaudeAgentOptions(
            system_prompt="claude_code",
            permission_mode="bypassPermissions"
        )
    ):
        print_message(msg)

await task_details()

### When to Use Each Tool

| Tool | When to Use |
|------|-------------|
| `TaskList` | See overview of all tasks |
| `TaskGet` | Get full details of one task |
| `TaskCreate` | Start planning work |
| `TaskUpdate` | Change status, add dependencies |

## Best Practices

### Task Creation
- Write clear, actionable subjects (imperative form: "Analyze files")
- Include detailed descriptions with acceptance criteria
- Always provide `activeForm` (present continuous: "Analyzing files")
- Break large tasks into smaller, testable chunks

### Dependencies
- Only add dependencies when truly required
- Use parallel tasks when possible for efficiency
- Document why dependencies exist in descriptions

### Status Management
- Always mark tasks `in_progress` before starting work
- Only mark `completed` when fully done (don't mark on errors)
- Use `TaskList` after updates to verify state

### Metadata
- Store custom data in task metadata (cost estimates, priority, tags)
- Use metadata for filtering and reporting
- Keep metadata JSON-serializable

## Exercises

Practice your task management skills:

### Exercise 1: Test-Driven Development Workflow
Create a TDD workflow with tasks for:
1. Write failing test
2. Implement minimal code to pass test
3. Refactor code
4. Run all tests

Set up dependencies so each step blocks the next.

### Exercise 2: Data Pipeline
Create a data processing pipeline with parallel tasks:
- Three parallel tasks: Extract data from source A, B, C
- One task: Transform all data (blocked by all extract tasks)
- One task: Load to database (blocked by transform)

### Exercise 3: Progress Tracking
Create 5 tasks, then:
1. Mark task 1 as `in_progress`
2. Complete task 1
3. Start and complete task 2
4. Check task list to see progress
5. Get detailed info on remaining tasks

In [ ]:
# Exercise 1: Your solution here


In [ ]:
# Exercise 2: Your solution here


In [ ]:
# Exercise 3: Your solution here


## Key Takeaways

- **Task tools help structure complex workflows** into manageable steps
- **Four core tools**: `TaskCreate`, `TaskUpdate`, `TaskList`, `TaskGet`
- **Dependencies control execution order** using `blockedBy` relationships
- **Status lifecycle**: `pending` → `in_progress` → `completed`
- **Parallel tasks** improve efficiency when work is independent
- **Metadata** enables custom tracking and reporting

Next up: **Hooks** - learn how to intercept and customize agent behavior at key lifecycle points!